In [2]:
# prompt: create a machine learning model that takes an existing dataset of songs with various characteristics and suggests songs for user based on a song request

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from pathlib import Path


In [3]:
# # Load the dataset (replace 'songs.csv' with your actual file)
file_path = Path('Resources/spotify_dataset.csv')
songs_df = pd.read_csv(file_path)

In [4]:
column_names = songs_df.columns.tolist()
column_names

['track',
 'artist',
 'uri',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'chorus_hit',
 'sections',
 'popularity',
 'decade']

In [7]:
songs_df['decade'] = songs_df['decade'].replace({
    '60s': 1960.0,
    '70s': 1970.0,
    '80s': 1980.0,
    '90s': 1990.0,
    '00s': 2000.0,  # we chose to do the full decade instead of 60.0, 70.0, etc because turning 00s into 00.0 could lead to confusion
    '10s': 2010.0
})

songs_df['decade'] = songs_df['decade'].astype(float)

/var/folders/gj/34pvzc9x2x98g0vpglqmrsz80000gn/T/ipykernel_80326/1750691394.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  songs_df['decade'] = songs_df['decade'].replace({


In [6]:
print(songs_df['decade'].unique())

['60s' '70s' '80s' '90s' '00s' '10s']


In [9]:
# # Preprocessing: Handle missing values and convert categorical features (if needed)
# # Example: Fill missing values in 'danceability' with the mean
# # songs_df['danceability'].fillna(songs_df['danceability'].mean(), inplace=True)

# Select relevant features for the model
features = ['danceability', 'energy', 'key','loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','popularity','decade']
X = songs_df[features]



In [10]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split the data (not strictly needed for KNN, but good practice)
X_train, X_test, _, _ = train_test_split(X_scaled, songs_df['track'], test_size=0.2, random_state=42)

In [11]:
# Train a k-Nearest Neighbors model
model = NearestNeighbors(n_neighbors=10, algorithm='auto') # Adjust n_neighbors as needed
model.fit(X_train)

NearestNeighbors(n_neighbors=10)

In [12]:
def recommend_songs(song_id, num_recommendations=5):
    try:
        # Find the index of the requested song
        song_index = songs_df[songs_df['track'] == song_id].index[0]
        # Get the features for the requested song
        song_features = X_scaled[song_index].reshape(1,-1)

        # Find the nearest neighbors
        distances, indices = model.kneighbors(song_features)
        # Exclude the song itself
        recommended_indices = indices.flatten()[1:]
        # Print recommendations
        print(f"Recommendations for {songs_df.loc[song_index, 'artist']} ({song_id}):")
        for i in recommended_indices[:num_recommendations]:
            print(f"- {songs_df.loc[i, 'track']} ({songs_df.loc[i,'artist']})")
    except IndexError:
        print(f"Error: Song with ID '{song_id}' not found in the dataset.")


# Example usage (Replace 'song_id' with an actual ID)
recommend_songs('New Rules')

Recommendations for Dua Lipa (New Rules):
- Reminiscing (The Exboyfriends)
- Nieskonczenie maly (Tomasz Stanko Quintet)
- Susie Darlin' (Tommy Roe)
- Let Me Make Love To You (The O'Jays)
- Children's Playtime (Tony Kinsey)


In [13]:
recommend_songs('XO')

Recommendations for John Mayer (XO):
- Sexy Eyes (Dr. Hook)
- Try To Remember (Ed Ames)
- I Love That Swamp Pop Music (The Boogie Kings)
- You're No Good (Linda Ronstadt)
- Check Yo Self (Ice Cube Featuring Das EFX)


In [14]:
recommend_songs('Classical Gas')

Recommendations for Mason Williams (Classical Gas):
- I Didn't Know I Loved You (Till I Saw You Rock And Roll) (Gary Glitter)
- The Floor (Johnny Gill)
- Godaari Gattundi (P. Susheela)
- Mandrill (Mandrill)
- Raz tak, raz nie (Adam Makowicz)


In [15]:
recommend_songs('Subterranean Homesick Blues')

Recommendations for Bob Dylan (Subterranean Homesick Blues):
- Sorairo Days (Shoko Nakagawa)
- These Are The Times (Dru Hill)
- Chicane Destination (999)
- Moon And Sand (Gil Evans)
- With Pen In Hand (Vikki Carr)
